<a href="https://colab.research.google.com/github/Paaaark/Kaggle/blob/main/titanic/features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
import csv
from tabulate import tabulate

In [49]:
%run /content/drive/MyDrive/Kaggle/titanic/helper.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
origin = pd.read_csv('/content/drive/MyDrive/Kaggle/titanic/train.csv')
origin.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Clean the data and Create new variables

In [45]:
df = pd.DataFrame(origin)
df_y = df['Survived']
df = df.drop(['PassengerId', 'Ticket', 'Cabin', 'Survived'], axis=1)

df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = 1
df['IsAlone'].loc[df['FamilySize'] > 1] = 0
df['Title'] = df['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]

df = df.drop(['Name'], axis=1)

df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode())
df['Age'] = df['Age'].fillna(df['Age'].median())

df['FareBin'] = pd.qcut(df['Fare'], 4)
df['AgeBin'] = pd.qcut(df['Age'], 4)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Pclass      891 non-null    int64   
 1   Sex         891 non-null    object  
 2   Age         891 non-null    float64 
 3   SibSp       891 non-null    int64   
 4   Parch       891 non-null    int64   
 5   Fare        891 non-null    float64 
 6   Embarked    889 non-null    object  
 7   FamilySize  891 non-null    int64   
 8   IsAlone     891 non-null    int64   
 9   Title       891 non-null    object  
 10  FareBin     891 non-null    category
 11  AgeBin      891 non-null    category
dtypes: category(2), float64(2), int64(5), object(3)
memory usage: 71.9+ KB


<ipython-input-45-77de31158c0b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IsAlone'].loc[df['FamilySize'] > 1] = 0


In [46]:
label = LabelEncoder()

df['Sex'] = label.fit_transform(df['Sex'])
df['Embarked'] = label.fit_transform(df['Embarked'])
df['FareBin'] = label.fit_transform(df['FareBin'])
df['AgeBin'] = label.fit_transform(df['AgeBin'])
df['Title'] = label.fit_transform(df['Title'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    int64  
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Embarked    891 non-null    int64  
 7   FamilySize  891 non-null    int64  
 8   IsAlone     891 non-null    int64  
 9   Title       891 non-null    int64  
 10  FareBin     891 non-null    int64  
 11  AgeBin      891 non-null    int64  
dtypes: float64(2), int64(10)
memory usage: 83.7 KB


In [47]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Title,FareBin,AgeBin
0,3,1,22.0,1,0,7.2500,2,2,0,11,0,0
1,1,0,38.0,1,0,71.2833,0,2,0,12,3,3
2,3,0,26.0,0,0,7.9250,2,1,1,8,1,1
3,1,0,35.0,1,0,53.1000,2,2,0,12,3,2
4,3,1,35.0,0,0,8.0500,2,1,1,11,1,2


In [52]:
X, y = process_and_engineer_data(origin, columns=['Pclass', 'Sex', 'Embarked', 'FamilySize', 'IsAlone', 'Title', 'FareBin', 'AgeBin'])
train_x, test_x, train_y, test_y = model_selection.train_test_split(X, y, random_state = 0)

print(train_x.shape)

(668, 8)


<ipython-input-49-8414925b950a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IsAlone'].loc[df['FamilySize'] > 1] = 0
